# Preâmbulo

Imports, funções, downloads e instalação do Pytorch.

In [0]:
 # Basic imports.
import os
import time
import numpy as np
import torch

from torch import nn
from torch import optim

from torch.utils.data import DataLoader
from torch.utils import data
from torch.backends import cudnn

from torchvision import models
from torchvision import datasets
from torchvision import transforms

from skimage import io

from sklearn import metrics

from matplotlib import pyplot as plt

%matplotlib inline

cudnn.benchmark = True

In [0]:
# Setting predefined arguments.
args = {
    'epoch_num': 10,      # Number of epochs.
    'n_classes': 10,      # Number of classes.
    'lr': 0.0005,         # Learning rate.
    'weight_decay': 1e-5, # L2 penalty.
    'num_workers': 3,     # Number of workers on data loader.
    'batch_size': 100,    # Mini-batch size.
    'print_freq': 1,      # Printing frequency.
    'lambda_s': 2.0, #0.2,      # Sparsity importance in loss computation.
}

if torch.cuda.is_available():
    args['device'] = torch.device('cuda')
else:
    args['device'] = torch.device('cpu')

print(args['device'])

# Carregando o  MNIST

In [0]:
# Root directory for the dataset (to be downloaded).
root = './'

# Transformations over the dataset.
data_transforms = transforms.Compose([
    transforms.ToTensor()
])

# Setting datasets and dataloaders.
train_set = datasets.MNIST(root,
                           train=True,
                           download=True,
                           transform=data_transforms)
test_set = datasets.MNIST(root,
                          train=False,
                          download=False,
                          transform=data_transforms)

# Random subset of training data for small data scenarios.
# torch.random.manual_seed(12345)
# indices = torch.randperm(len(train_set))[:500]
# train_set = data.Subset(train_set, indices)

# Setting dataloaders.
train_loader = DataLoader(train_set,
                          args['batch_size'],
                          num_workers=args['num_workers'],
                          shuffle=True)
test_loader = DataLoader(test_set,
                         args['batch_size'],
                         num_workers=args['num_workers'],
                         shuffle=False)

# Printing training and testing dataset sizes.
print('Size of training set: ' + str(len(train_set)) + ' samples')
print('Size of test set: ' + str(len(test_set)) + ' samples')

# AutoEncoder Esparso

[Regularizações](https://medium.com/datadriveninvestor/l1-l2-regularization-7f1b4fe948f2) são componentes comuns em vários métodos da área de Machine Learning que podem servir como um viés para que o algoritmo dê preferência a soluções mais simples, potencialmente prevenindo overfitting no caso de modelos **overcomplete** e/ou **small data**. A loss de um Sparse AE (SAE) adiciona um termo de regularização $\mathcal{L}_{s}$ à loss de regressão $\mathcal{L}_{r}$ de um AE tradicional. Dessa forma, a loss total $\mathcal{L}_{t}$ é dada por:

$\mathcal{L}_{t}(x, \hat{x}, z) = \mathcal{L}_{r}(x, \hat{x}) + \lambda_{s} \mathcal{L}_{s}(z).$

Um AE tradicional produz features com ativações consideravelmente densas dos inputs passados a ele, já que, como o objetivo principal é reconstrução, toda informação possível deve ser mantida nas representações latentes da rede.

![Dense AE](https://www.dropbox.com/s/nfiix8cfk5g9wue/Sparse_AE_1.png?dl=1)

Em contraponto, SAEs produzem representações esparsas dos dados que podem ser utilizadas para realizar [**Sparse Coding**](https://en.wikipedia.org/wiki/Sparse_dictionary_learning), o que tem várias aplicações dentro da área de Machine Learning, incluindo [melhorias na performance de algumas tarefas de classificação](https://arxiv.org/pdf/1312.5663.pdf). A imagem abaixo mostra uma rede com ativações mais esparsas devido à adição de um termo de regularização $\mathcal{L}_{s}(z)$.

![Sparse AE](https://www.dropbox.com/s/rs27590a80srntp/Sparse_AE_2.png?dl=1)

Para mais informações sobre **Sparse Coding** (e também outros tópicos interessantes de Machine Learning), um bom material pode ser encontrado nos seguintes vídeos:
*   https://www.youtube.com/watch?v=7a0_iEruGoM
*   https://www.youtube.com/watch?v=L6qhzWWtqQs

# Atividade Prática: modificando o AE Convolucional para criar representações esparsas

1.   Crie uma função que calcule o componente $\mathcal{L}_{s}(z)$, recebendo o output do encoder computado pelo AE Convolucional e aplique uma regularização L1 sobre ele: $\frac{1}{N}\sum_{i=0}^{N}{|z_{i}|}$;
2.   Utilize o fator $\lambda_{s}$ (*args\['lambda_s\']*) para criar uma loss composta $\mathcal{L}_{t}$ nas funções *train()* e *test()* com a loss de regressão $\mathcal{L}_{r}$ tradicional do AE, a qual já está implementada. A loss final $\mathcal{L}_{t}$ é que deve ser usada para computar o backward nas funções de treino e teste;
3.   Compare visualmente a densidade de ativações nos feature maps calculados no AE Convolucional denso (implementado na aula passada) e no AE Convolucional esparso.

PS.: Compute $\mathcal{L}_{s}$ usando apenas operações vetoriais, ou seja, sem o uso de um *for*, o que deixaria muito lento o algoritmo.